In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import cv2  # Assuming you have OpenCV installed
import torchvision
from torchvision.datasets import ImageFolder
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import itertools
import math
from vit_pytorch.efficient import ViT

In [2]:
# Required Parameters
dataset = "C:\\Study\\OJT\\Dataset\\UCF50"          # Dataset path
dataset2 = "C:\\Study\\OJT\\Dataset_Extraction\\UCF50\\Full"                # Dataset2 path
train_path = "C:\\Study\\OJT\\Dataset_Extraction\\UCF50\\UCF50Train"             # Training path
test_path = "C:\\Study\\OJT\\Dataset_Extraction\\UCF50\\UCF50Test"             # Testing path
no_of_frames = 10000                     # Total number of frames to be extracted
categories = os.listdir(dataset)        # Name of each class/category

In [3]:
# Creating dataset directory
try:
    os.mkdir(dataset2)
    print("Folder {} created...".format(dataset2))
except:
    print("A folder {} already exists...".format(dataset2))

A folder C:\Study\OJT\Dataset_Extraction\UCF50\Full already exists...


In [4]:
# Creating training_set directory
try:
    os.mkdir(train_path)
    print("Folder {} created...".format(train_path))
except:
    print("A folder {} already exists...".format(train_path))

A folder C:\Study\OJT\Dataset_Extraction\UCF50\UCF50Train already exists...


In [5]:
# Creating testing_set directory
try:
    os.mkdir(test_path)
    print("Folder {} created...".format(test_path))
except:
    print("A folder {} already exists...".format(test_path))

A folder C:\Study\OJT\Dataset_Extraction\UCF50\UCF50Test already exists...


In [6]:
# Creating same directories for dataset2/ that are already present in the dataset directory
for category in categories:
    try:
        os.mkdir(os.path.join(dataset2, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, dataset))

Folder BaseballPitch created...
Folder Basketball created...
Folder BenchPress created...
Folder Biking created...
Folder Billiards created...
Folder BreastStroke created...
Folder CleanAndJerk created...
Folder Diving created...
Folder Drumming created...
Folder Fencing created...
Folder GolfSwing created...
Folder HighJump created...
Folder HorseRace created...
Folder HorseRiding created...
Folder HulaHoop created...
Folder JavelinThrow created...
Folder JugglingBalls created...
Folder JumpingJack created...
Folder JumpRope created...
Folder Kayaking created...
Folder Lunges created...
Folder MilitaryParade created...
Folder Mixing created...
Folder Nunchucks created...
Folder PizzaTossing created...
Folder PlayingGuitar created...
Folder PlayingPiano created...
Folder PlayingTabla created...
Folder PlayingViolin created...
Folder PoleVault created...
Folder PommelHorse created...
Folder PullUps created...
Folder Punch created...
Folder PushUps created...
Folder RockClimbingIndoor cr

In [7]:
for category in categories:
    try:
        os.mkdir(os.path.join(train_path, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, train_path))


A folder already exists, named BaseballPitch...
A folder already exists, named Basketball...
A folder already exists, named BenchPress...
A folder already exists, named Biking...
A folder already exists, named Billiards...
A folder already exists, named BreastStroke...
A folder already exists, named CleanAndJerk...
A folder already exists, named Diving...
A folder already exists, named Drumming...
A folder already exists, named Fencing...
A folder already exists, named GolfSwing...
A folder already exists, named HighJump...
A folder already exists, named HorseRace...
A folder already exists, named HorseRiding...
A folder already exists, named HulaHoop...
A folder already exists, named JavelinThrow...
A folder already exists, named JugglingBalls...
A folder already exists, named JumpingJack...
A folder already exists, named JumpRope...
A folder already exists, named Kayaking...
A folder already exists, named Lunges...
A folder already exists, named MilitaryParade...
A folder already exi

In [8]:
# Creating same directories for testing_set/ that are already present in the dataset directory
for category in categories:
    try:
        os.mkdir(os.path.join(test_path, category))
        print("Folder {} created...".format(category))
    except:
        print("A folder already exists, named {}...".format(category, test_path))


A folder already exists, named BaseballPitch...
A folder already exists, named Basketball...
A folder already exists, named BenchPress...
A folder already exists, named Biking...
A folder already exists, named Billiards...
A folder already exists, named BreastStroke...
A folder already exists, named CleanAndJerk...
A folder already exists, named Diving...
A folder already exists, named Drumming...
A folder already exists, named Fencing...
A folder already exists, named GolfSwing...
A folder already exists, named HighJump...
A folder already exists, named HorseRace...
A folder already exists, named HorseRiding...
A folder already exists, named HulaHoop...
A folder already exists, named JavelinThrow...
A folder already exists, named JugglingBalls...
A folder already exists, named JumpingJack...
A folder already exists, named JumpRope...
A folder already exists, named Kayaking...
A folder already exists, named Lunges...
A folder already exists, named MilitaryParade...
A folder already exi

In [9]:
# Combining multiple videos into single video file
for category in tqdm(categories):
    videofiles = glob.glob(os.path.join(dataset, category, "**/*.avi"), recursive=True)
    if videofiles:
        cap = cv2.VideoCapture(videofiles[0])
    else:
        print("No video files found in {}/{}".format(dataset, category))
    video_index = 0
    cap = cv2.VideoCapture(videofiles[0])    
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    # fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter("{}/{}/{}.avi".format(dataset2, category, category), fourcc, 25, (320, 240))
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            video_index += 1
            if video_index >= len(videofiles):
                break
            else:
                cap = cv2.VideoCapture(videofiles[ video_index ])
                ret, frame = cap.read()
                out.write(frame)
        else:
            out.write(frame)
    cap.release()
    out.release()
    

  0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
# Saving total no. of frames of each classes/categories into an array
total_frames = []
for category in tqdm(categories):
    cap = cv2.VideoCapture(dataset2 + "/" + category + "/" + category + ".avi")
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_frames.append(length)
    cap.release()
    
print(total_frames)

  0%|          | 0/50 [00:00<?, ?it/s]

[14944, 18949, 18966, 32839, 42936, 18340, 21216, 26284, 46503, 14570, 23444, 13796, 32746, 38943, 13084, 11855, 30354, 10387, 64620, 30829, 32488, 15983, 16307, 35875, 13988, 39933, 23076, 34262, 18383, 26672, 40890, 16769, 43334, 8721, 56045, 21399, 45329, 29090, 16486, 31370, 22995, 45578, 23160, 18256, 26493, 12609, 24002, 12836, 27697, 23350]


In [11]:
# Extracting one frame per five frame from the Videos
for category in tqdm(categories):
    count = 0    
    a = glob.glob(dataset2 + '/' + category + '/' + category +'.avi')
    for i in range(len(a)):
        cap = cv2.VideoCapture(a[i])
        frameRate = cap.get(5)
        while(cap.isOpened()):
            frameId = cap.get(1)
            ret, frame = cap.read()
            if (ret != True):
                break
            if (frameId % math.floor(frameRate) == 0):
                cv2.imwrite(train_path + '/' + category + '/{}_{}.jpg'.format(category, count), frame)
                count += 1
        cap.release()

  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
# Moving 150 random images from training_set into testing_set
for category in tqdm(categories):
    sub_file = [file for file in glob.glob(train_path +'/'+ category +'/'+ "*.jpg")]
    test_files = random.sample(sub_file, 25)
    for test_file in test_files:
        img = cv2.imread(test_file)
        os.remove(test_file)
        test_filename = os.path.basename(test_file)
        cv2.imwrite(test_path +'/' + category + '/' + test_filename , img)
        

  0%|          | 0/50 [00:00<?, ?it/s]

In [26]:
# Training settings
batch_size = 64
epochs = 20
lr = 0.001
gamma = 0.7
seed = 42

In [15]:
# Define the transformation for loading and preprocessing images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),  # You can adjust the crop size as needed
    transforms.ToTensor(),
])

In [16]:
# Create a custom dataset for the entire training set
full_dataset = ImageFolder(train_path, transform=transform)

In [17]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))  # 80% for training
val_size = len(full_dataset) - train_size  # 20% for validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [32]:
# Create custom datasets for training and test
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

In [19]:
# Create data loaders for training and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [20]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

In [24]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=50,
    transformer=efficient_transformer,
    channels=3,
)

In [30]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [31]:
# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if batch_idx % 10 == 9:  # Print every 10 batches
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {running_loss / 10:.4f}, Training Accuracy: {100 * correct / total:.2f}%")
            running_loss = 0.0

    scheduler.step()

    # Validation accuracy calculation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}, Training Accuracy: {100 * correct / total:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

# Evaluation on the test set
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_outputs = model(test_inputs)
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()

print(f"Test Accuracy: {100 * test_correct / test_total:.2f}%")
torch.save(model.state_dict(), 'ViT_UCF50_pth')

Epoch 1, Batch 10/812, Loss: 3.9607, Training Accuracy: 2.66%
Epoch 1, Batch 20/812, Loss: 3.7782, Training Accuracy: 4.30%
Epoch 1, Batch 30/812, Loss: 3.6191, Training Accuracy: 5.78%
Epoch 1, Batch 40/812, Loss: 3.4880, Training Accuracy: 7.58%
Epoch 1, Batch 50/812, Loss: 3.3489, Training Accuracy: 9.06%
Epoch 1, Batch 60/812, Loss: 3.3231, Training Accuracy: 9.87%
Epoch 1, Batch 70/812, Loss: 3.2074, Training Accuracy: 11.03%
Epoch 1, Batch 80/812, Loss: 3.0624, Training Accuracy: 12.05%
Epoch 1, Batch 90/812, Loss: 2.9883, Training Accuracy: 13.32%
Epoch 1, Batch 100/812, Loss: 2.9204, Training Accuracy: 14.47%
Epoch 1, Batch 110/812, Loss: 2.9466, Training Accuracy: 15.28%
Epoch 1, Batch 120/812, Loss: 2.7122, Training Accuracy: 16.39%
Epoch 1, Batch 130/812, Loss: 2.7586, Training Accuracy: 17.31%
Epoch 1, Batch 140/812, Loss: 2.6899, Training Accuracy: 18.20%
Epoch 1, Batch 150/812, Loss: 2.5355, Training Accuracy: 19.28%
Epoch 1, Batch 160/812, Loss: 2.6229, Training Accuracy